In [19]:
import os

# GLOVE_DIR = './glove.6B/'
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 100

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# data = pd.read_csv('../input/spam.csv',encoding='latin-1')
data = pd.read_csv('./spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics

num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
# mat_texts = tok.texts_to_matrix(texts,mode='count')

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(texts,tags, test_size = 0.3)
# mat_texts_tr = tok.texts_to_matrix(x_train,mode='count')
# mat_texts_tst = tok.texts_to_matrix(x_test,mode='count')

max_len = MAX_SEQUENCE_LENGTH
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)

word_index = tok.word_index

cnn_texts_mat = sequence.pad_sequences(x_train,maxlen=max_len)
# max_len = 100
cnn_texts_mat_tst = sequence.pad_sequences(x_test,maxlen=max_len)

In [22]:
c = 0
for word, index in word_index.items():
    c += 1
    if c <20:
        print(word, index)

i 1
to 2
you 3
a 4
the 5
u 6
and 7
in 8
is 9
me 10
my 11
for 12
your 13
it 14
of 15
call 16
have 17
on 18
2 19


In [27]:
len(word_index)

8920

In [26]:
# dictionary of just words were using
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    if word in word_index.keys():
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()
c = 0
for word, word_vector in embeddings_index.items():
    c += 1
    if c <= 5:
        print(word, word_vector)
# print(embeddings_index)
print('Found %s word vectors.' % len(embeddings_index))

the [-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.270

In [55]:
# dict of whole whole glove
# embeddings_index = {}
# f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

In [28]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [30]:
embedding_matrix[0:2]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [57]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [58]:
def check_model(model,xtr,ytr,xts,yts):
    model.fit(xtr,ytr,batch_size=32,epochs=10,verbose=1,validation_split=0.3)
    print(' ')
    model.evaluate(xts,yts)

In [62]:
from keras.layers import LSTM
def get_LSTM_v4():    
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
#     model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])

    print('Train...')
    return model

In [63]:
m = get_LSTM_v4()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst ,y_test)

Train...
Train on 2730 samples, validate on 1170 samples
Epoch 1/10
2730/2730 [==============================] - 10s 4ms/step - loss: 0.3504 - acc: 0.8989 - binary_accuracy: 0.8989 - val_loss: 0.3203 - val_acc: 0.9231 - val_binary_accuracy: 0.9231
Epoch 2/10
2730/2730 [==============================] - 8s 3ms/step - loss: 0.2563 - acc: 0.9403 - binary_accuracy: 0.9403 - val_loss: 0.1757 - val_acc: 0.9444 - val_binary_accuracy: 0.9444
Epoch 3/10
2730/2730 [==============================] - 8s 3ms/step - loss: 0.2105 - acc: 0.9553 - binary_accuracy: 0.9553 - val_loss: 0.1473 - val_acc: 0.9530 - val_binary_accuracy: 0.9530
Epoch 4/10
2730/2730 [==============================] - 8s 3ms/step - loss: 0.1628 - acc: 0.9619 - binary_accuracy: 0.9619 - val_loss: 0.1451 - val_acc: 0.9470 - val_binary_accuracy: 0.9470
Epoch 5/10
2730/2730 [==============================] - 8s 3ms/step - loss: 0.1320 - acc: 0.9560 - binary_accuracy: 0.9560 - val_loss: 0.1319 - val_acc: 0.9530 - val_binary_accuracy:

In [61]:
len(word_index) + 1

8921